In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
import scipy.optimize

In [2]:
#gradient descent method
def gdm(learningR,toleranceV,theta,thetaOld,n_classes,input_size,lambda_,input_data,labels):
    while (np.linalg.norm(theta - thetaOld,ord = 2) >= toleranceV):
        thetaOld = theta
        cost,derivative = softmax_cost(theta, n_classes, input_size, lambda_, input_data, labels)
        theta = thetaOld - learningR * derivative
    return theta

In [3]:
def softmax_cost(theta, n_classes, input_size, lambda_, data, labels):
    """
    Compute the cost and derivative.
    n_classes: the number of classes
    input_size: the size N of the input vector
    lambda_: weight decay parameter
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    labels: an M x 1 matrix containing the labels corresponding for the input data
    """
    # k stands for the number of classes
    # n is the number of features and m is the number of samples
    k = n_classes
    n, m = data.shape
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Matrix of indicator fuction with shape (k, m)
    labels = np.reshape(labels,(m,))
    indicator = scipy.sparse.csr_matrix((np.ones(m), (labels, np.arange(m))))
    indicator = np.array(indicator.todense())
    # Cost function
    cost = -1.0/m * np.sum(indicator * np.log(proba)) + 0.5*lambda_*np.sum(theta*theta)
    # Gradient matrix with shape (k, n)
    grad = -1.0/m * (indicator - proba).dot(data.T) + lambda_*theta
    # Unroll the gradient matrices into a vector
    grad = grad.ravel()
    return cost, grad

In [57]:
def softmax_train(input_size, n_classes, lambda_, input_data, labels, options={'maxiter': 400, 'disp': True}):
    # initialize parameters
    theta = 0.005 * np.random.randn(n_classes * input_size)
    thetaOld = theta * 1000

    # Find out the optimal theta
    opt_theta =  gdm(0.01,0.01,theta,thetaOld,n_classes,input_size,lambda_,input_data,labels)
    opt_theta = np.array(opt_theta)
    model = {'opt_theta': opt_theta, 'n_classes': n_classes, 'input_size': input_size}
    return model

In [58]:
def softmax_predict(model, data):
    """
    model: model trained using softmax_train
    data: the N x M input matrix, where each column data[:, i] corresponds to
          a single test set
    pred: the prediction array.
    """
    theta = model['opt_theta'] # Optimal theta
    k = model['n_classes']  # Number of classes
    n = model['input_size'] # Input size (number of features)
    # Reshape theta
    theta = theta.reshape((k, n))
    # Probability with shape (k, m)
    theta_data = theta.dot(data)
    alpha = np.max(theta_data, axis=0)
    theta_data -= alpha # Avoid numerical problem due to large values of exp(theta_data)
    proba = np.exp(theta_data) / np.sum(np.exp(theta_data), axis=0)
    # Prediction values
    pred = np.argmax(proba, axis=0)
    return pred

In [59]:
X = loadmat('TrainImages.mat')
X = X['trainData'].T
y = loadmat('TrainLabels.mat')
y = y['trainLabels']
X_test = loadmat('TestImages.mat')
X_test = X_test['testData'].T
y_test = loadmat('TestLabels.mat')
y_test = y_test['testLabels']
print X.shape
print X_test.shape



(784, 60000)
(784, 10000)


In [60]:
input_size = 28 * 28 # Size of input vector (MNIST images are 28x28)
n_classes = 10       # Number of classes (MNIST images fall into 10 classes)
lambda_ = 1e-4 # Weight decay parameter
print lambda_

0.0001


In [61]:
options = {'maxiter': 100, 'disp': True}
model = softmax_train(input_size, n_classes, lambda_, X, y, options)

In [62]:
pred = softmax_predict(model, X_test)
y_test = y_test.tolist()
pred = pred.tolist()
print accuracy_score(y_test,pred)

0.6413


In this program, we used a gradient descent method for finding the optimum theta. We tried out various combination of the tolerance values, each of which gave us different accuracy. It was observed that as we go on decreasing the tolerance value, our accuracy was increasing. The following table summarizes the accuracy that we got when we went on decreasing the tolerance value. It was also observed that gradient descent method was taking more time to find the optimum theata for small values of tolerance values.

|learning rate|tolerance Value|Accuracy|
|:---:|:---:|:---:|
|0.01|0.01|64.13|
|0.01|0.09|75.37|
|0.01|0.006|80.46|
|0.01|0.003|84.43|
|0.01|0.001|88.00|
|0.01|0,0009|88.14|
|0.01|0.0006|89.23|
|0.01|0.0003|90.51|

The following table compares the accuracy of model using gradient descent method and L-BFGS-B optimising method:

|Model|Accuracy|
|:---:|:---:|
|GDM| |90.17|
|L-BFGS| |92.58|